# Demo: Zonal Statistics with Multispectral Imagery & Wildfire Perimeters

**ABT/HYD 182 – Lab 8 Demo**

This demo uses **publicly available data** (no Box/Canvas):
- **Raster**: Sentinel-2 L2A multispectral imagery from [Microsoft Planetary Computer](https://planetarycomputer.microsoft.com/)
- **Vector**: California wildfire perimeters from [California Open Data](https://data.ca.gov/) / [CAL FIRE GIS](https://data.cnra.ca.gov/dataset/cal-fire-gis)

You will run the same concepts as Lab 8: read raster, overlay vector, compute NDVI, segment vegetation, and run **zonal statistics** (e.g., mean NDVI per fire polygon).

## Table of Contents

1. [Setup & install](#section1)
2. [Load multispectral (Sentinel-2) data](#section2)
3. [Visualize RGB](#section3)
4. [Load wildfire perimeters (vector)](#section4)
5. [Overlay vector on raster](#section5)
6. [Compute NDVI](#section6)
7. [Mask vegetation](#section7)
8. [Zonal statistics](#section8)
9. [Merge and summarize](#section9)

---
<a name="section1"></a>
## 1. Setup & install
---

Install and import: `rasterio`, `geopandas`, `rasterstats`, `pystac-client`, `planetary-computer`.

In [ ]:
# Run once in Colab or your environment
!pip install -q rasterio geopandas rasterstats pystac-client planetary-computer

In [ ]:
import rasterio
from rasterio.windows import from_bounds
from rasterio.plot import reshape_as_image
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from rasterstats import zonal_stats
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from pystac_client import Client
import planetary_computer

---
<a name="section2"></a>
## 2. Load multispectral (Sentinel-2) data
---

We search the [Planetary Computer](https://planetarycomputer.microsoft.com/) STAC API for **Sentinel-2 L2A** over a small area in **California** (e.g., near wildfire-prone region or Central Valley). We then read **red**, **green**, **blue**, and **NIR** bands and stack them into a single raster array with a common transform (needed for NDVI and zonal stats).

In [ ]:
# Area of interest: California (lon, lat)
# Example: area near Sacramento/Davis (can change to overlap known fires)
minx, miny = -122.0, 38.4
maxx, maxy = -121.6, 38.7
bbox = (minx, miny, maxx, maxy)

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime="2023-08-01/2023-09-30",  # summer, good for vegetation/fire
    query={"eo:cloud_cover": {"lt": 20}}
)
items = list(search.items())
if not items:
    raise RuntimeError("No Sentinel-2 scenes found for this bbox/date. Try different bbox or datetime.")
item = items[0]
print(f"Using scene: {item.id}")

In [ ]:
# Band names in Sentinel-2 L2A (visual and NIR)
band_names = ["red", "green", "blue", "nir"]
asset_keys = ["B04", "B03", "B02", "B08"]  # red, green, blue, NIR

def read_band(item, key, bbox):
    href = planetary_computer.sign_url(item.assets[key].href)
    with rasterio.open(href) as src:
        # bbox is lon/lat (WGS84); transform to raster CRS for from_bounds
        from rasterio.warp import transform_bounds
        bbox_proj = transform_bounds("EPSG:4326", src.crs, bbox[0], bbox[1], bbox[2], bbox[3])
        window = from_bounds(*bbox_proj, src.transform)
        data = src.read(1, window=window)
        transform = src.window_transform(window)
        return data, transform, src.crs

# Read first band to get transform and crs
red, transform, crs = read_band(item, "B04", bbox)
shape = red.shape

# Read other bands with same window (same bbox)
green, _, _ = read_band(item, "B03", bbox)
blue, _, _ = read_band(item, "B02", bbox)
nir, _, _ = read_band(item, "B08", bbox)

# Stack into (bands, rows, cols) for compatibility with lab
image_array = np.stack([blue, green, red, np.zeros_like(red), nir], axis=0)
# For Sentinel-2 we have 4 bands here: B, G, R, NIR (no red-edge; use 0 as placeholder)
print("image_array shape (bands, rows, cols):", image_array.shape)
print("CRS:", crs)
print("Transform:", transform)

---
<a name="section3"></a>
## 3. Visualize RGB
---

In [ ]:
# Reshape to (rows, cols, bands) for display
image = reshape_as_image(image_array)
# RGB = bands 0,1,2 (B,G,R); clip and normalize for display
rgb = image[:, :, :3].astype(float)
rgb = np.clip(rgb / (np.nanpercentile(rgb, 98) or 1), 0, 1)

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(rgb)
ax.set_title("Sentinel-2 RGB (California AOI)")
ax.axis("off")
plt.tight_layout()
plt.show()

---
<a name="section4"></a>
## 4. Load wildfire perimeters (vector)
---

We load **California fire perimeters** from a public URL. If the full dataset is large, we clip to our raster bounding box. If no fires intersect the scene, we create **sample zones** (rectangles) so zonal statistics still run.

In [ ]:
# California fire perimeters (GeoJSON). Alternative: Recent Large Fires only.
# If this URL is slow or fails, use a local file or the "Recent Large Fire Perimeters" dataset.
fire_perimeters_url = (
    "https://gis.data.cnra.ca.gov/api/download/v1/items/e3802d2abf8741a187e73a9db49d68fe/geojson?layers=0"
)

try:
    gdf_all = gpd.read_file(fire_perimeters_url, bbox=bbox)
except Exception as e:
    print("Could not load from URL:", e)
    gdf_all = gpd.GeoDataFrame()

# If we have no geometries, create 3 sample zones (boxes) inside our raster extent
from shapely.geometry import box

with rasterio.open("MEM", "w", driver="GTiff", height=shape[0], width=shape[1], count=1, dtype=red.dtype, crs=crs, transform=transform) as mem:
    bounds = mem.bounds

if len(gdf_all) == 0:
    # Create 3 rectangular zones for demo
    L, B, R, T = bounds
    w, h = (R - L) / 4, (T - B) / 4
    zones = [
        box(L, B, L + w, B + h),
        box(L + w, B + h, L + 2 * w, B + 2 * h),
        box(L + 2 * w, B + 2 * h, R, T),
    ]
    gdf = gpd.GeoDataFrame({"zone_id": [1, 2, 3], "geometry": zones}, crs=crs)
    print("Using 3 sample zones (no fire perimeters in bbox)")
else:
    gdf = gdf_all.to_crs(crs)
    # Clip to raster bounds to avoid huge polygons
    gdf = gpd.clip(gdf, box(*bounds))
    if len(gdf) == 0:
        zones = [box(bounds[0], bounds[1], bounds[0] + (bounds[2]-bounds[0])/3, bounds[3]),
                 box(bounds[0] + (bounds[2]-bounds[0])/3, bounds[1], bounds[2], bounds[3])]
        gdf = gpd.GeoDataFrame({"zone_id": [1, 2], "geometry": zones}, crs=crs)
        print("No fires in extent; using 2 sample zones")
    else:
        gdf["zone_id"] = range(len(gdf))
        print(f"Loaded {len(gdf)} fire perimeter(s)")

print(gdf.head())

---
<a name="section5"></a>
## 5. Overlay vector on raster
---

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
extent = (transform.c, transform.c + shape[1] * transform.a, transform.f + shape[0] * transform.e, transform.f)
ax.imshow(rgb, extent=extent)
gdf.plot(ax=ax, facecolor="none", edgecolor="red", linewidth=2)
ax.set_title("RGB with zone boundaries (fire perimeters or sample zones)")
ax.set_xlim(transform.c, transform.c + shape[1] * transform.a)
ax.set_ylim(transform.f + shape[0] * transform.e, transform.f)
ax.set_aspect("equal")
plt.tight_layout()
plt.show()

---
<a name="section6"></a>
## 6. Compute NDVI
---

$$NDVI = \frac{(NIR - Red)}{(NIR + Red)}$$

In [ ]:
red_f = red.astype(np.float64)
nir_f = nir.astype(np.float64)
ndvi = np.where(
    (nir_f + red_f) != 0,
    (nir_f - red_f) / (nir_f + red_f),
    np.nan
)
# Optional: replace invalid with 0 for zonal_stats
ndvi_safe = np.nan_to_num(ndvi, nan=0.0, posinf=0.0, neginf=0.0)

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
im = ax.imshow(ndvi, cmap="RdYlGn", vmin=-0.2, vmax=0.8)
plt.colorbar(im, ax=ax, label="NDVI")
ax.set_title("NDVI")
ax.axis("off")
plt.tight_layout()
plt.show()

---
<a name="section7"></a>
## 7. Mask vegetation
---

Segment vegetation by thresholding NDVI (e.g. NDVI > 0.4) and optionally show masked RGB.

In [ ]:
ndvi_threshold = 0.4
mask = ndvi_safe > ndvi_threshold

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(mask, cmap="gray")
axes[0].set_title(f"NDVI mask (threshold = {ndvi_threshold})")
axes[0].axis("off")

mask_3d = np.dstack([mask] * 3)
segmented = mask_3d * rgb
axes[1].imshow(segmented)
axes[1].set_title("Masked RGB (vegetation only)")
axes[1].axis("off")
plt.tight_layout()
plt.show()

---
<a name="section8"></a>
## 8. Zonal statistics
---

Compute **min**, **max**, **median**, **mean**, and **sum** of NDVI for each zone (fire polygon or sample box).

In [ ]:
stats = ["min", "max", "median", "mean", "sum"]
ndvi_stats = zonal_stats(
    gdf,
    ndvi_safe,
    affine=transform,
    stats=stats,
    all_touched=False
)
df_ndvi_stats = pd.DataFrame(ndvi_stats)
print("NDVI zonal statistics:")
df_ndvi_stats

---
<a name="section9"></a>
## 9. Merge and summarize
---

Merge the zonal stats back to the GeoDataFrame (like Lab 8) and show mean NDVI per zone.

In [ ]:
gdf_merged = pd.merge(gdf, df_ndvi_stats, how="left", left_index=True, right_index=True)
print(gdf_merged[[c for c in gdf_merged.columns if c != "geometry"]].head(10))

if "mean" in gdf_merged.columns:
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    gdf_merged.plot(column="mean", ax=ax, legend=True, cmap="RdYlGn", legend_kwds={"label": "Mean NDVI"})
    ax.set_title("Mean NDVI by zone")
    ax.axis("off")
    plt.tight_layout()
    plt.show()

---
## Summary

This demo replicated the Lab 8 workflow using **public data**:
- **Raster**: Sentinel-2 L2A from Planetary Computer (red, green, blue, NIR).
- **Vector**: California fire perimeters (or synthetic zones if none in extent).
- **Indices**: NDVI and optional vegetation mask.
- **Zonal statistics**: min, max, median, mean, sum of NDVI per polygon.
- **Merge**: Attach stats to the vector layer.

You can change `bbox` and `datetime` to explore other regions or years; use "Recent Large Fire Perimeters" for a smaller vector file if the full perimeter dataset is slow.